# <center><u>Données massives

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

In [3]:
Data=pd.read_excel(r"C:\Users\User_K\OneDrive\Desktop\credit_risk_dataset.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User_K\\OneDrive\\Desktop\\credit_risk_dataset.xlsx'

In [ ]:
Data

In [ ]:
Data.columns

In [ ]:
Data.shape

In [ ]:
Data.dtypes

In [ ]:
Data.isna()

In [ ]:
missing_rate=Data.isna().sum()/Data.shape[0]

In [ ]:
missing_rate

In [ ]:
plt.figure(figsize=(15,6))
sns.heatmap(Data.isna())

On remarque qu'on a des données manquants au niveau de la variable "person_emp_length" qui refere aux nombres d'années d'emploie avec un pourcentage de 3% à peu pres ainsi que 9.5% à peu pres des données de la variable "loan_int_rate".

-Puisque on a une grande base de données et que le pourcentage des données manquants au niveau de la variable "person_emp_length" on va supprimer ces données manquants .

In [ ]:
vars=['Index','loan_int_rate']
dt1= pd.DataFrame(data=Data,columns=vars)

In [ ]:
dt1

In [ ]:
Data.drop('loan_int_rate',1,inplace=True)

In [ ]:
Data=Data.dropna()

In [ ]:
Data.isna().sum()

Notre Base est maintenant sans valeurs manquante mais aussi sans la variale 'loan_int_rate'. On a maintenant merger notre colonne et la base en utlisant Index comme cle. Puis on va essayer de traiter les valeurs manquantes au niveau de cette variable qui reste puis on va 

In [ ]:
data=pd.merge(Data,dt1,on='Index',how='inner')

In [ ]:
data

In [ ]:
imputer = KNNImputer(n_neighbors=2)

In [ ]:
le = LabelEncoder()

In [ ]:
data['person_home_ownership']=le.fit_transform(data['person_home_ownership'])
data['loan_intent']=le.fit_transform(data['loan_intent'])
data['loan_grade']=le.fit_transform(data['loan_grade'])
data['cb_person_default_on_file']=le.fit_transform(data['cb_person_default_on_file'])

In [ ]:
data.dtypes

In [ ]:
data

In [ ]:
data1 = pd.DataFrame(imputer.fit_transform(data),columns = data.columns)

In [ ]:
data1['loan_int_rate']

    Maintenant qu'on a plus de valeurs manquantes, on va passer au traitement des valeurs aberantes 

In [ ]:
data1[data1['person_age']<data1['person_emp_length']]

In [ ]:
data1[data1['person_age']<23]['person_emp_length']

In [ ]:
data1[data1['person_age']<23]['person_emp_length'].median()

In [ ]:
#EOn a pris la mediane parceque dans la liste qu'on a utilisé on a 2 valeurs aberrantes donc on va remplacer avec 4 :
data1['person_emp_length'][0] = 4
data1['person_emp_length'][209] = 4

In [ ]:
data1[data1['person_emp_length']>40]

à partir de plus de 40 ans de carrière, peut être moins plausible, mais dans ce cas, plausible car âge de la personne est  78 ans

### Maintenant que notre base de données est clean, on pourra commencer la construction de notre modèle. mais avant on va checker le desequilibre de notre base de données

In [ ]:
data1['loan_status'].value_counts()

In [ ]:
df_y = data1[['loan_status']]
df_y

In [ ]:
df_x = data1[['person_age','person_income','person_home_ownership','person_emp_length','loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length']]
df_x

avant de faire de l'oversampling ou undersampling, il faut séparer échantillon test et train car ces 2 méthodes  doivent se      faire uniquement sur l'échantillon train

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size = 0.2, random_state = 42)

In [ ]:
y_train['loan_status'].value_counts()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
x_train_ros, y_train_ros= ros.fit_resample(x_train, y_train)

In [ ]:
x_train_ros

In [ ]:
y_train_ros['loan_status'].value_counts()

l'échantillon maintenant équilibré, on va pouvoir passer aux modèles

In [ ]:
x_train_ros.to_excel("x_train_ros.xlsx")

In [ ]:
y_train_ros.to_excel("y_train_ros.xlsx")

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn import model_selection,linear_model, metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb
import seaborn as sns

In [ ]:
def model_assess(model, name='Default'):
    model.fit(x_train_ros, y_train_ros)
    preds = model.predict(x_test)
    preds_proba = model.predict_proba(x_test)
    print(name, '\n',classification_report(y_test, model.predict(x_test)))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=150)
model_assess(knn, name='KNN')

In [ ]:
lg = LogisticRegression(random_state=42)
model_assess(lg, 'Logistic Regression')

In [ ]:
D_tree = DecisionTreeClassifier(max_depth=10, min_samples_split=2, min_samples_leaf=1, random_state=42)
model_assess(D_tree, 'DecisionTree Classifier')

In [ ]:
xgb = xgb.XGBClassifier(objective="binary:logistic", random_state=42) 
model_assess(xgb, 'XGBoost')

In [ ]:
names = [
    "Logistic Regression",
    "Support Vector Machine",
    "Neural Network",
    "Random Forest",
    "XGBoost",
    "KNeighborsClassifier"
]

In [ ]:
models = [
    LogisticRegression(),
    SVC(),
    MLPClassifier(),
    RandomForestClassifier(),
    xgboost.XGBRFClassifier(max_depth=5),
    KNeighborsClassifier(n_neighbors=8, p=2)
]

In [ ]:
accuracy=[]
for model, name in zip(models,names):
    model.fit(x_train_ros, y_train_ros)
    
    y_pred = model.predict(x_test)
    print('Confusion matrix of ',name)
    print(confusion_matrix(y_test, y_pred))
    acc_score = accuracy_score(y_test, y_pred)
    print('Accuracy score is ',acc_score)
    accuracy.append(acc_score)

In [ ]:
df_accuracy=pd.DataFrame(zip(names, accuracy),columns =['Model', 'Accuracy']).sort_values("Accuracy",ascending=False)
df_accuracy

In [ ]:
fig = plt.figure(figsize = (20, 5))

plt.bar(df_accuracy["Model"], df_accuracy["Accuracy"])
plt.xlabel("Models",fontsize=16)
plt.ylabel("Respective Accuracy",fontsize=16)
plt.title("Sorted Accuracy of different Models", fontsize = 20)
plt.xticks(fontsize = 11, horizontalalignment = 'right', rotation = 30)
plt.yticks(fontsize = 13)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=50, max_features="auto", random_state=44)
rf_model.fit(x_train_ros, y_train_ros)
predictions = rf_model.predict(x_test)
predictions

In [ ]:
rf_model.predict_proba(x_test).head(20)

In [ ]:
for i in df.columns:
    print("The importance of feature "i" is "{round(rf_model.feature_importances_[i] * 100, 2)}"%.")
